In [9]:
import os
import pandas as pd
import numpy as np
import json
import pickle
import argparse

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel, permutation_test
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from statsmodels.stats import multitest
from matplotlib.backends.backend_pdf import PdfPages

from tfsemb_class import tsne, save_pickle, add_speech
from tfsplt_encoding import get_cmap_smap, aggregate_data, organize_data, get_elecbrain
from tfsplt_utils import read_folder, load_pickle, get_con_color
from tfsplt_brainmap import get_sigelecs, Colorbar, make_brainmap
from tfsplt_brainmap_cat import make_brainmap_cat

def fdr(pvals):
    _, pcor, _, _ = multitest.multipletests(
        pvals, method="fdr_bh", is_sorted=False
    )
    return pcor

### Load Data

In [ ]:
class Args(argparse.Namespace):
  sid = ["777"] # subjects
  project = "podcast"
  formats = [ # encoding folder
    "../data/encoding/podcast/20240208-rephael/kw-podcast-full-777-glove50-lag2k-25-all-rephael1/*/*%s.csv",
    "../data/encoding/podcast/20240208-rephael/kw-podcast-full-777-glove50-lag2k-25-all-rephael2/*/*%s.csv",
  ]
  labels = [
    "aggregate",
    "control",
  ]
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  keys = ["comp"] # comprehension and/or production
  lags_plot = np.arange(-2000,2001,25) # encoding lags
  lags_show = np.arange(-2000,2001,25) # lags for the effect
  lc_by = "labels"
  ls_by = "keys"

# Aggregate Data
args = Args()
args.unique_labels = list(dict.fromkeys(args.labels))
args.unique_keys = list(dict.fromkeys(args.keys))
args.lags_show = args.lags_show / 1000
args.lags_plot = args.lags_plot / 1000
args = get_sigelecs(args)  # get significant electrodes

df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary
df

### Encoding Plot

In [14]:
colors = ["red","blue"]
figsize = (10,5)

fig, ax = plt.subplots(figsize=figsize)
for color, (map_key, subdf) in zip(colors, df.groupby(["label"], axis=0)):
    vals = subdf.mean(axis=0)
    err = subdf.sem(axis=0)
    label = map_key
    ax.fill_between(
        args.lags_show,
        vals - err,
        vals + err,
        alpha=0.2,
        color=color,
    )
    ax.plot(
        args.lags_show,
        vals,
        label=f"{label} ({len(subdf)})",
        color=color,
        ls="-",
    )
ax.axhline(0, ls="dashed", alpha=0.3, c="k")
ax.axvline(0, ls="dashed", alpha=0.3, c="k")
ax.legend(loc="upper right", frameon=False)
ax.set(xlabel="Lag (s)", ylabel="Correlation (r)", title="Global average")
pdf = PdfPages("../aggregate.pdf")
pdf.savefig(fig)
plt.close()
plt.savefig("../aggregate.jpeg")

<Figure size 640x480 with 0 Axes>

In [15]:
for (electrode, sid), subdf in df.groupby(["electrode", "sid"], axis=0):
    fig, ax = plt.subplots(figsize=figsize)
    for (label, _, _, _), values in subdf.iterrows():
        if label == "aggregate":
            color = "red"
        else:
            color = "blue"
        ax.plot(
            args.lags_show,
            values,
            label=label,
            color=color,
            ls="-",
        )
    ax.axhline(0, ls="dashed", alpha=0.3, c="k")
    ax.axvline(0, ls="dashed", alpha=0.3, c="k")
    ax.set_ylim(-0.1,0.5)
    ax.legend(loc="upper left", frameon=False)
    ax.set(
        xlabel="Lag (s)",
        ylabel="Correlation (r)",
        title=f"{sid} {electrode}",
    )
    imname = get_elecbrain(electrode)
    if os.path.isfile(imname):
        arr_image = plt.imread(imname, format="png")
        fig.figimage(
            arr_image,
            fig.bbox.xmax - arr_image.shape[1],
            fig.bbox.ymax - arr_image.shape[0],
            zorder=5,
        )
    pdf.savefig(fig)
    plt.close()
pdf.close()

### Add max and area

In [ ]:
df2 = df.copy()
df2["max"] = df2.loc[:,(np.arange(0,161))].max(axis=1)
df2.reset_index(drop=False, inplace=True)

area_df = pd.read_csv("../data/plotting/paper-mia/area_electrodes2.csv")
df2 = df2.merge(area_df, on="electrode",how="left")
df2

### Max Corr

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "both" # only plot left hemisphere
  outfile = "../podcast_%s.png"
  final = False
  final2 = False
  shiny = False

args = Args()

args.color_split = [Colorbar(title="best layer",colorscale="viridis",bar_max=0.4, bar_min=0)]

df_plot = df2.copy()
df_plot["effect"] = df_plot["max"]
df_plot["subject"] = df_plot.electrode.str.split("_", n=1, expand=True)[0]
for model, subdf in df_plot.groupby("label", axis=0):
    subdf_plot = subdf.loc[:, ("subject", "electrode", "effect")]
    print(model, subdf_plot.effect.mean())
    fig = make_brainmap(args, subdf_plot, args.outfile % model) # plot png

### Max Corr Diff

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "both" # only plot left hemisphere
  outfile = "../podcast-diff-%s.png"
  final = False
  final2 = False
  shiny = False

args = Args()

# Colorsplit
pos_bar = Colorbar(title=f"Δ % corr pos",colorscale=[[0, "rgb(255,248,240)"], [1, "rgb(255,0,0)"]],bar_min=0,bar_max=0.025)
neg_bar = Colorbar(title=f"Δ % corr neg",colorscale=[[0, "rgb(0,0,255)"], [1, "rgb(240,248,255)"]],bar_min=-0.025,bar_max=0)
args.color_split = [neg_bar,0,pos_bar]

df_plot = df2.copy()
df_base = df_plot.loc[df_plot["label"] == "control",("electrode","max")].copy()
df_plot = df_plot.merge(df_base,left_on="electrode",right_on="electrode",how="outer")
# df_plot["effect"] = (df_plot["max_x"] - df_plot["max_y"]) / df_plot[["max_x", "max_y"]].max(axis=1)
# df_plot["effect"] = df_plot["max_x"] / df_plot["max_y"] * 100 - 100
df_plot["effect"] = df_plot["max_x"] - df_plot["max_y"]
df_plot["subject"] = df_plot.electrode.str.split("_", n=1, expand=True)[0]
for model, subdf in df_plot.groupby("label", axis=0):
  if model == "control":
    continue
  subdf_plot = subdf.loc[:, ("subject", "electrode", "effect")]
  print(model, subdf_plot.effect.describe())
  fig = make_brainmap(args, subdf_plot, args.outfile % model) # plot png